<a href="https://colab.research.google.com/github/yoyoFC/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
G

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-13 00:10:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2021-12-13 00:10:43 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [16]:
# Create the customers_table DataFrame
customers_df = df.groupby('customer_id').agg({'customer_id':'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             2|
|   44524872|             1|
|   16711087|             1|
|   14116630|             3|
|   10548283|             1|
|   12879980|            15|
|   11913481|             1|
|   49875798|             3|
|   40993003|             1|
|    6272214|             3|
|     417296|             1|
|   36524024|             2|
|   16146148|             1|
|   43274367|             1|
|   38692392|            14|
|   24359722|             1|
|   45450058|             5|
|    1960118|             1|
|   36980552|             1|
|   10200363|             2|
+-----------+--------------+
only showing top 20 rows



In [19]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates(subset=['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0188399518|Planetwise Flanne...|
|0310986524|God Bless Baby Re...|
|0448437104|Dick and Jane Rea...|
|0448448564|The Ultimate Dick...|
|0615447279|Stop Pacifier Suc...|
|0670062049|5 Pink Gumdrops +...|
|0671493183|Doggies (Boynton ...|
|070539462X|C.R. GIBSON ABC &...|
|0741631016|Twelve Piece Baby...|
|0857579940|Gruffalos Child S...|
|097293751X|Baby Tracker® - D...|
|0977905497|Potty Training Ch...|
|0980027594|Nature's Lullabie...|
|0983676534|The Letterheads -...|
|098955791X|The Best Paper Ai...|
|1059879875|Modern House Find...|
|141279000X|Grandma's Little ...|
|1432109529|"My First Year" M...|
|1442498145|Chicka Chicka Abc...|
|1450739865|Father Football G...|
+----------+--------------------+
only showing top 20 rows



In [20]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

In [22]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df .show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
|R13EPSFP5DODN5|          4|            0|          0|   N|                Y|
| R6RBP4HTE67SY|          5|            0|          0|   N|                Y|
|R15B3EU40RSU2W|          5|            0|          0|   N|                Y|
| RP4DD53A4ZJA2|          5|            0|          0|   N|                Y|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [23]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cgq2ewzuuqs1.us-east-2.rds.amazonaws.com:5432/postgres"
#PASSWORD REMOVE FOR SAFETY REASONS
config = {"user":"postgres", 
          "password": "-------", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [26]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [28]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [36]:
###################################################################
#DELIVERABLE 2

#Filter 1:  total_votes >= 20
df_Reviews_Filter1 = vine_df[vine_df['total_votes'] >= 20]
df_Reviews_Filter1.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [42]:
#Filter 2: From previous dataframe divide helpful_votes divided by total_votes > 50%
df_Reviews_Filter2 = df_Reviews_Filter1.withColumn('hel/total',df_Reviews_Filter1['helpful_votes']/df_Reviews_Filter1['total_votes'])
df_Reviews_Filter2.show()


+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           hel/total|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|  0.9120879120879121|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|                 1.0|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|  0.9869451697127938|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|  0.9642857142857143|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|  0.8723404255319149|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|  0.8333333333333334|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|  0.9512195121951219|
|R3OIDSQJ8

In [44]:
df_Reviews_Filter2 = df_Reviews_Filter2[df_Reviews_Filter2 ['hel/total']>=0.5]
df_Reviews_Filter2.show() 

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         hel/total|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|0.9120879120879121|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|               1.0|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|0.9869451697127938|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|0.9642857142857143|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|0.8723404255319149|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|0.8333333333333334|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|0.9512195121951219|
|R3OIDSQJ84W7J1|          5|  

In [45]:
#Filter 3: Create a new data frame with the review written under the Vine program 
df_Reviews_Filter3 = df_Reviews_Filter2[df_Reviews_Filter2['vine']=='Y']
df_Reviews_Filter3.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         hel/total|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|0.8723404255319149|
| RSA6JQ346JZHZ|          5|           55|         64|   Y|                Y|          0.859375|
|R1FXF4HRMCLG4C|          5|           69|         84|   Y|                N|0.8214285714285714|
| RCTBWC3II42MG|          4|          113|        117|   Y|                N|0.9658119658119658|
| RTMQM2CQ1XIZ0|          5|           21|         25|   Y|                N|              0.84|
|R2CBJLCKQ612KU|          4|           27|         35|   Y|                N|0.7714285714285715|
|R322QDGO4AV2B5|          5|           55|         65|   Y|                N|0.8461538461538461|
| R2S7CXRDGQ6EE|          4|  

In [46]:
#Filter 4: Create a new data frame with review written out of the Vine program 
df_Reviews_Filter4 = df_Reviews_Filter2[df_Reviews_Filter2['vine']=='N']
df_Reviews_Filter4.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         hel/total|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|0.9120879120879121|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|               1.0|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|0.9869451697127938|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|0.9642857142857143|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|0.8333333333333334|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|0.9512195121951219|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|             0.875|
| RODE8K12S7148|          5|  

In [56]:
#Filter 5 : VINE PROGRAM, How many reviews, 5 start and percentage of 5 start
total_rev_VINE_Y = df_Reviews_Filter3.count()
total_5_VINE_Y = df_Reviews_Filter3[(df_Reviews_Filter3['star_rating']==5)].count()
print("VINE PROGRAM")
print("The total number of review is: %f" %total_rev_VINE_Y)
print("The total number of 5 start review is: %f" %total_5_VINE_Y)
print("Percentage of 5 start review: %f" %(total_5_VINE_Y*100/total_rev_VINE_Y))

VINE PROGRAM
The total number of review is: 463.000000
The total number of 5 start review is: 202.000000
Percentage of 5 start review: 43.628510


In [57]:
#Filter 6 : NO VINE PROGRAM, How many reviews, 5 start and percentage of 5 start
total_rev_VINE_N = df_Reviews_Filter4.count()
total_5_VINE_N = df_Reviews_Filter4[(df_Reviews_Filter4['star_rating']==5)].count()
print("NO VINE PROGRAM")
print("The total number of review is: %f" %total_rev_VINE_N)
print("The total number of 5 start review is: %f" %total_5_VINE_N)
print("Percentage of 5 start review: %f" %(total_5_VINE_N*100/total_rev_VINE_N))

NO VINE PROGRAM
The total number of review is: 25094.000000
The total number of 5 start review is: 12033.000000
Percentage of 5 start review: 47.951702
